In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Connect to the SQLite database (since it's in the same folder, you can just use the filename)
db_path = 'metadata-110mil.sqlite'
conn = sqlite3.connect(db_path)

# Check available tables in the database (optional)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Available tables:", tables)

# Specify the table you want to load
table_name = 'metadata'  # Replace with your actual table name

# Load the table into a DataFrame
df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)

# Close the database connection
conn.close()

# Display the first few rows of the DataFrame
df.head()


Available tables:        name
0  metadata


In [ ]:
test = df.iloc[0]
print(test)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Set the font to a universal one like Noto Sans
#rcParams['font.sans-serif'] = ['Noto Sans', 'DejaVu Sans', 'Arial Unicode MS']  # Ensure Noto Sans is installed
#rcParams['axes.unicode_minus'] = False  # Avoid issues with minus signs

# Load your data (assuming it's already in a DataFrame `df`)
# df = pd.read_csv('your_file.csv')  # Uncomment this if you're loading from a CSV file

# Split categories by commas and stack them to get a single column of all categories
#all_categories = df['Category'].str.split(',').explode()

# Strip any extra whitespace from each category (important if there are spaces after commas)
#all_categories = all_categories.str.strip()

# Count occurrences of each unique category
#category_counts = all_categories.value_counts()

# Filter categories to include only those with at least 500 occurrences
#filtered_category_counts = category_counts[category_counts >= 10000]

# Plot as a bar chart
#plt.figure(figsize=(20, 6))
#filtered_category_counts.plot(kind='bar', color='skyblue')
#plt.title('Distribution of Categories (at least 10000 entries)')
#plt.xlabel('Category')
#plt.ylabel('Count')
#plt.xticks(rotation=90, ha='right')
#plt.tight_layout()
#plt.show()


![category_distribution](category_distribution.png)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Take a random sample of 100,000 rows (adjust the number as needed for performance)
#sample_df = df['word_count'].sample(100000, random_state=42)

#plt.figure(figsize=(10, 6))
#sns.histplot(sample_df, bins=50, kde=True)

#plt.xlabel("Word Count")
#plt.ylabel("Frequency")
#plt.title("Distribution of Word Count (Sampled)")

#plt.show()


![word distribution](word_distribution.png)

<h2> Cleaning and Formating Data: </h2>

In [ ]:
df.info()

In [ ]:
missing_all_values = df[df["Publisher"] == ""].value_counts()#.index.tolist()
len(missing_all_values) # TODO: Clean all that are missing. 79 missing is rows that dont hae any value beside path

In [ ]:
rows_with_null = (df.isnull()).sum()
rows_with_null

In [ ]:
rows_with_missing_values = (df.isnull() | (df == "")).any(axis=1).sum()
rows_with_missing_values

In [ ]:
rows_with_missing_values / len(df) * 100

In [ ]:
all_missing = (df == "").sum()
all_missing

In [ ]:
# Changing data type for chapter and word count into int type

df['word_count'] = pd.to_numeric(df['word_count'], errors='coerce').astype('Int64')
df['chapter_count'] = pd.to_numeric(df['chapter_count'], errors='coerce').astype('Int64')

In [ ]:
#df_usable = df.copy(deep=True) 

# Remove redundant  and unimportant columns

df_usable = df.drop(columns=['Chapters', 'Words','Path','Story URL','Author URL'])

# remove rows where all values are missing (79 of those)

df_usable = df_usable[df_usable['word_count'] != '']

# All where summary, genre and category is missing

df_usable = df_usable[df_usable['Summary'] != '']
df_usable = df_usable[df_usable['Category'] != '']
df_usable = df_usable[df_usable['Genre'] != '']

# Removing extreme word count values  (100 < x < 2,000,000)

df_usable = df_usable[(df_usable['word_count'] > 100) & (df_usable['word_count'] < 2000000)]
df_usable=df_usable[(df_usable['Language'] == 'English')]

In [ ]:
# Counting after cleaning all the missing
missing = (df_usable == "").sum()
missing

In [ ]:
df_usable.columns

In [ ]:
len(df_usable)

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Ensure nltk resources are downloaded
import nltk
nltk.download('punkt')  # Tokenizer data
nltk.download('wordnet')  # Lemmatizer data
nltk.download('stopwords')  # Stopwords data
nltk.download('omw-1.4')  # WordNet data
nltk.download('punkt_tab')

In [ ]:
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
custom_stop_words = set(stopwords.words('english')).union(ENGLISH_STOP_WORDS)

# Preprocessing function
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabet characters
    text = text.lower()  # Lowercase text
    tokens = word_tokenize(text)  # Tokenize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in custom_stop_words]  # Lemmatize and remove stopwords
    return tokens

# Apply preprocessing to Title and Summary
df_usable['processed_title'] = df_usable['Title'].apply(preprocess_text)
df_usable['processed_summary'] = df_usable['Summary'].apply(preprocess_text)

# Combine title and summary
df_usable['combined_text'] = df_usable['processed_title'] + df_usable['processed_summary']

# Group by Category and Genre
grouped_data = df_usable.groupby(['Category', 'Genre'])

# Collect unique words for each group
unique_words_by_group = defaultdict(list)

for (category, genre), group in grouped_data:
    all_words = [word for text in group['combined_text'] for word in text]
    unique_words = set(all_words)  # Find unique words
    unique_words_by_group[(category, genre)] = unique_words

# Convert to DataFrame for easier analysis
unique_words_df = pd.DataFrame([
    {'Category': k[0], 'Genre': k[1], 'Unique_Words': list(v)} 
    for k, v in unique_words_by_group.items()
])

In [ ]:
unique_words_df

In [ ]:
unique_words_df[unique_words_df["Unique_Words"].apply(len) == 0]

In [ ]:
df_usable[df_usable["Category"].str.contains("Harry", case=False, na=False)]

<h2> Build a model </h2>

In [ ]:
df_usable["Category"].value_counts()

In [ ]:
df_usable["word_count"].sum()

In [ ]:
numerical_columns = df[['word_count', 'chapter_count']].copy()

# Convert these columns to numeric (if needed)
numerical_columns = numerical_columns.apply(pd.to_numeric, errors='coerce')

# Drop any rows with NaN values in numerical columns to avoid calculation issues
numerical_columns = numerical_columns.dropna()

# Calculate the correlation matrix
correlation_matrix = numerical_columns.corr()

# Plot the correlation matrix
plt.figure(figsize=(8, 6))
plt.matshow(correlation_matrix, cmap='coolwarm', fignum=1)
plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation=90)
plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)
plt.colorbar()
plt.title("Correlation Matrix", pad=20)
plt.show()

correlation_matrix

In [ ]:
df.shape

In [ ]:
df['word_count'] = pd.to_numeric(df['word_count'], errors='coerce').astype('Int64')
df[df["word_count"] > 2000000]

In [ ]:
df.iloc[5568444]["Summary"]

In [ ]:
df["Status"].value_counts()

In [ ]:
#print(len(all_categories.unique()))#9975

In [ ]:
df.iloc[12430]['Summary']
df.iloc[12430]

In [ ]:
# Count occurrences in the "Category" column
category_counts = df_usable['Category'].value_counts()
valid_categories = category_counts[category_counts >= 100].index
df_filtered = df_usable[df_usable['Category'].isin(valid_categories)]
df_usable=df_filtered
print(len(df_usable['Category'].unique()))

In [ ]:
df_usable.head()
df_usable = df_usable.drop(columns=['Packaged', 'Publisher'])


In [ ]:
df_usable.shape
# Specify the output file
output_file = "filtered_usable_df.csv"

# Save to CSV
df_usable.to_csv(output_file, sep=";", encoding="utf-8", index=False)  # Set index=False to avoid saving the index

print(f"DataFrame successfully written to {output_file}")

In [ ]:
df_usable2 = pd.read_csv("filtered_usable_df.csv", sep=";")
print(df_usable2.shape)

In [ ]:
df_usable.head()